In [1]:
import pandas as pd
import seaborn as sns

from utils.tags import tag_elements

In [2]:
# 'https://drive.google.com/uc?id=1_osI1ZMtR3chrKtwRfwlNyrr5VP0OIT0'

article_data = pd.read_csv('./datasets/article_about_data.gz',
                           compression='gzip',
                           parse_dates=['datetime'],
                           index_col='id')

article_data

,datetime,tags,article_lenght,rate,views,bookmarks,comments,header
id,,,,,,,,
1,2006-07-13 14:23:39,"['хабрахаб','wiki','faq','чаво','механизм','дв...",428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра
2,2006-07-13 20:45:38,"['ошибки','баги','bugs','error','warning','habr']",216.0,0,18000,1,37,Мы знаем много недоделок на сайте… но!
4,2006-07-14 13:59:37,"['google','маслов','сокур и партнеры','pr','со...",258.0,1,4000,1,15,"Маслов, Сокур и партнеры пиарят Google в России"
6,2006-07-14 17:19:18,"['хабрахабр','подкасты','mp3','контент','текст...",362.0,0,4500,1,35,Подкасты на Хабрахабре
7,2006-07-14 21:48:02,"['логин','login','habr']",190.0,0,3500,1,11,Самопроизвольное разлогинивание
...,...,...,...,...,...,...,...,...
128530,2011-09-15 15:49:53,"['windows 8','разработка под windows']",120.0,63,4800,12,151,Краткий видео-обзор Windows 8
128531,2011-09-15 15:52:15,"['','блог компании google developers']",1147.0,1,1000,0,0,Расширенные возможности интернет-магазина Chro...
128532,2011-09-15 18:42:06,"['социальные сети','образование','обучение онл...",4391.0,20,2000,29,25,Концепция образовательной социальной сети


__Предобработка данных__

_Предобработка тэгов_

In [3]:
# Теги загружены в виде строки вида ['tag_1', 'tag_1', 'tag_1'],
# привожу его к массиву.

article_data.tags = [tag for tag in article_data.tags.str.replace(
                     pat="[\[\]']", repl='', regex=True).str.split(',') if tag != '']


In [4]:
# Раскрываю датафрейм по колонке tags. Применяю к каждому отдельному тэгу tag_elements,
# которая очищает тэг и разделяет его на отдельные слова (массив слов).
# Повторно раскрываю датафрейм по колонке tags.

article_data = article_data.explode('tags')
article_data.tags = article_data.tags.apply(tag_elements)
article_data = article_data.explode('tags')

article_data

,datetime,tags,article_lenght,rate,views,bookmarks,comments,header
id,,,,,,,,
1,2006-07-13 14:23:39,хабрахаб,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра
1,2006-07-13 14:23:39,wiki,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра
1,2006-07-13 14:23:39,faq,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра
1,2006-07-13 14:23:39,чаво,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра
1,2006-07-13 14:23:39,механизм,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра
...,...,...,...,...,...,...,...,...
128534,2011-09-15 19:33:21,статический,10249.0,40,12000,32,34,Анализ утилит статического анализа C++ кода
128534,2011-09-15 19:33:21,анализ,10249.0,40,12000,32,34,Анализ утилит статического анализа C++ кода
128534,2011-09-15 19:33:21,кода,10249.0,40,12000,32,34,Анализ утилит статического анализа C++ кода


In [5]:
# Выбираю тэги и подсчитываю их встречаемость.
# Рассчитываю долю встречаемости конкретного тэга.

tags = article_data.groupby('tags', as_index=False) \
                   .agg({'comments': 'count'}) \
                   .rename(columns={'tags': 'tag', 'comments': 'used'}) \
                   .sort_values(['used', 'tag'])

total_used = tags.used.sum()

tags = tags.assign(used_perc = tags.used / total_used)

tags

,tag,used,used_perc
5,00000000,1,0.000002
6,000:,1,0.000002
7,000prm,1,0.000002
8,000psd,1,0.000002
9,001,1,0.000002
...,...,...,...
42159,разработка,5710,0.010997
8515,google,6786,0.013069
26710,блог,7160,0.013790
33589,компании,11551,0.022246


In [ ]:
# Какая-то полезная работа с тэгами...

In [6]:
# Совмещаю данные по тэгам с исходным датафреймом.

article_data = article_data.merge(tags, left_on='tags', right_on='tag', how='left') \
                           .drop(columns='tags') \
                           .dropna()

article_data

,datetime,article_lenght,rate,views,bookmarks,comments,header,tag,used,used_perc
0,2006-07-13 14:23:39,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра,хабрахаб,2.0,0.000004
1,2006-07-13 14:23:39,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра,wiki,96.0,0.000185
2,2006-07-13 14:23:39,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра,faq,29.0,0.000056
3,2006-07-13 14:23:39,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра,чаво,2.0,0.000004
4,2006-07-13 14:23:39,428.0,1,36000,10,67,Wiki-FAQ для Хабрахабра,механизм,6.0,0.000012
...,...,...,...,...,...,...,...,...,...,...
530477,2011-09-15 19:33:21,10249.0,40,12000,32,34,Анализ утилит статического анализа C++ кода,статический,42.0,0.000081
530478,2011-09-15 19:33:21,10249.0,40,12000,32,34,Анализ утилит статического анализа C++ кода,анализ,285.0,0.000549
530479,2011-09-15 19:33:21,10249.0,40,12000,32,34,Анализ утилит статического анализа C++ кода,кода,157.0,0.000302
530480,2011-09-15 19:33:21,10249.0,40,12000,32,34,Анализ утилит статического анализа C++ кода,проектирование,323.0,0.000622


__Это на потом, может быть использую.__

In [ ]:
# 'https://drive.google.com/uc?id=1YjYmulVuJPH0IP770enNPWo9WPHdf1U0'

# article_text = pd.read_csv('./datasets/article_text_data.gz',
#                            compression='gzip',
#                            index_col='id')
